# Checkpoints: TANF Exiters

Authors: Maryah Garner, Rukhshan Arif Mian

***Please work on this checkpoints notebook if you used TANF exiters (**tanf_end = 1**) to define your cohort in*** `02_Creating_a_cohort.ipynb`.

This notebook serves as an overview of what was discussed in `03_Linkage_and_Longitudinal_Analysis.ipynb` through **3 checkpoints**. 

At each checkpoint you will be replacing the `___` with the appropriate variable, function or R code snippet. 

Participants are encouraged to attempt the checkpoints on their own. Having said that, hints and suggested solutions are provided and these can be accessed by utilizing the following code:

Hints: `check_#.hint()`

Solutions: `check_#.solution()` – your solutions may vary based on how you define your cohort and the name of your cohort table. We have shared our suggested solutions.

In both cases, # refers to the checkpoint number. For example: we can access the hint and solution for Checkpoint 2 using: `check_2.hint()` and `check_2.solution` respectively. Note: Codes for accessing hints and solutions are currently commented out – in order for these to run, we would need to uncomment them first. 

**The purpose of this notebook and all other checkpoint notebooks is to get you to practice making changes to the code that will help construct your own research project. We have given hints and solutions but these are more applicable to a generic research project. You are encouraged to think about how these checkpoints, hints/solutions may help formulate and address your research question.**

In [ ]:
# Switching off warnings
options(warn = -1)

# Database interaction imports
suppressMessages(library(odbc))

# for data manipulation/visualization
suppressMessages(library(tidyverse))

# scaling data, calculating percentages, overriding default graphing
suppressMessages(library(scales))

# yearqtr format
suppressMessages(library(zoo))

#Switching on warnings
options(warn = 0)

source("03_Linkage_and_Longitudinal_Analysis_hints_solutions_Exiters.txt") # defining hints + solutions

In [ ]:
# Connect to the database
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

As a reminder, please continue working on this notebook **only if you used TANF exiters** to create your cohort. (If you used TANF enterers, you should work on the `03_Linkage_and_Longitudinal_Analysis_checkpoints_Enterers.ipynb` checkpoints notebook)


In `03_Linkage_and_Longitudinal_Analysis.ipynb`, we focused on linking our cohort to Indiana's wage records. We created employment and job-level measures based on this linked cohort-wage data. 

## Checkpoint 1: Linking Cohort with UI wages
<font color='red'>
For the first checkpoint, you will be linking your cohort to Indiana's wage records. By your cohort, we refer to the cohort that you created in 02_Creating_a_cohort_checkpoints.ipynb and saved as team#_cohort_name where # refers to your team number and name refers to your name. 
</font>


In `03_Linkage_and_Longitudinal_Analysis.ipynb`, we looked at one quarter before our start-date of interest and one quarter after our end-date of interest (Refer to the *Linking TANF, UI Wages, and Employer Industry Over Time* heading). As a reminder, we do this because we want information on employment transitions. For this checkpoint, we ask you to make 3 updates in order to link the cohort of your choice (that you created in `02_Creating_a_cohort_checkpoints.ipynb`) with Indiana's wage records:

1. Fill in the first blank with the name of your cohort that you created in `02_Creating_a_cohort_checkpoints.ipynb`
2. Fill in the blanks in line 6 with year-quarters (in the format: YYYY Q#. Example: 2017 Q2) you would like to analyze the wage data for your cohort of interest. Note that we included 6 blanks, but you can use as many or as few as you would like.
> You might want to look at employment outcomes for the year after exiting TANF, so you would select the year-quarter starting in the quarter of exit you used to define your cohort and end a year-quarter after exit. For example: In the `03_Linkage_and_Longitidunal_Analysis.ipynb` notebook, we are looking at employment outcomes for individuals who exited TANF during 2018 Q2 so we are pulling in the wage from 2018 Q2 throught 2019 Q3. If you want to look at longer run employment outcomes you will want to expand that time frame.

3. Fill in the last blank with the name of your cohort that you created in `02_Creating_a_cohort_checkpoints.ipynb`.

In [ ]:
qry <- "SELECT nb.*, wr.Empr_no, wr.Wage, wr.yr_quarter
    FROM (tr_tdc_2022.dbo.____ nb
    LEFT JOIN 
    (SELECT SSN, Empr_no, yr_quarter, Wage 
    FROM tr_tdc_2022.dbo.wages_tanf
    WHERE (yr_quarter IN ('___', '___', '___', '___', '___', '___')) 
    AND (SSN IN (SELECT DISTINCT SSN FROM tr_tdc_2022.dbo.____))) wr
    ON wr.SSN=nb.SSN)"

cohort_wages <- dbGetQuery(con, qry)

In [ ]:
# hint
check_1.hint()

In [ ]:
## solution: same as well
check_1.solution()

It may also be the case where you want to look at multiple years of wages. Please uncomment the code below and run it to view a suggested implementation that takes into account multiple years.

## Quarters Employed

Once we have our linked cohort, we want to look at the number of quarters employed for recipients who exited TANF. This is the number of quarters where an individual received a non-zero wage.

The first step to do so would be to get the total wages for an individual in each quarter. We approach this using the following code (no updates required): 

In [ ]:
# TOTAL WAGES for each quarter during your selected time-frame
tot_wages <- cohort_wages %>% 
    group_by(ssn, yr_quarter) %>% 
    summarise(totalwages=sum(Wage))

# view the first 6 outcomes
head(tot_wages)

## Checkpoint 2: Calculating Number of Quarters Employed

Next, we ask you to fill in the blank below to only keep entries **after** the first quarter of interest. The reason for excluding the quarter of exit is to look at all quarters **after** and individual exit TANF. Update the blank below with the quarter of exit you chose for your cohort. 

In [ ]:
tot_q_emp <- tot_wages %>% 
    # fill in the blank below with the year-quarter combination based on how you defined your cohort
    filter(yr_quarter != '___' | is.na(yr_quarter)) %>%
    group_by(ssn) %>%
    summarise(quarters_employed= sum(!is.na(totalwages)))

In [ ]:
# looking at the number of quarters employed post-exit
tot_q_emp %>%
    group_by(quarters_employed) %>%
    summarise(count_individuals = n())

In [ ]:
# hint
check_2.hint()

In [ ]:
# solution
check_2.solution()

# Employment Measures

In this section, we replicate the code from `03_Linkage_and_Longitudinal_Analysis.ipynb` based on your linked cohort. **Note: You do not have to make any changes to the code in this section as this is not a checkpoint. You will only need to run these.**

## Setup



In [ ]:
#Get time period from UI wage records (for each unique year quarter, generate a fixed time variable-easier to work with than dates)

#Pull out any DISTINCT year and quarter records from the wage tables and make sure that year is a string of length 4 
#(we want year to be 4 digits) and quarter is only a single digit.

wage_qry1 <- "
SELECT DISTINCT Year, Quarter, CAST(Year AS VARCHAR) + ' ' + 'Q' + CAST(QUARTER AS VARCHAR) as yr_quarter
FROM ds_in_fssa.dbo.wage_10pct
WHERE LEN(year)=4 AND LEN(Quarter)=1
ORDER BY Year, Quarter
"

#Order the unique year and quarter observations, create a variable yearq which joins together year and quarter (no spaces or other separators)
#Then cast that string to an integer, and create a time variable that enumerates by row number. 
year_q <-dbGetQuery(con, wage_qry1) %>% 
                       mutate(TIME=row_number()) %>% 
        select(yr_quarter, TIME)

In [ ]:
# Bring in that TIME variable from the year_q data frame
cohort_wages <- cohort_wages %>% 
    left_join(year_q, by = "yr_quarter") 

head(cohort_wages)

Next we want to join in the continuous time variable (TIME) to our frame. We use a left join on cohort_wages_industry (our cohort frame) using yearq to bring in the TIME variable. Once complete, you will have a dataframe that consists of all the wage records for the quarters you chose in Checkpoint 1. Recall that if a person from our cohort has no wages in this time period, they will have a single record missing year, quarter, and any other variables from the wage records but they will still exist as a unique ssn in the dataframe.

In [ ]:
#Create job-quarter level measures (will be aggregated to job-level measures)
emp_measures <- cohort_wages %>%
# Select variables that you need to measure employment 
    select(ssn, yr_quarter, Empr_no, TIME) %>% 
# To identify whether a person has the same employer, 
# from one month to the next, we will group by individuals and employers 
    group_by(ssn, Empr_no) %>%
# If an individual is only employed for one quarter, then they do not have steady employment (this also removes those not in wage records)
    filter(n() > 1) %>%
# Within each individual-employer grouping, sort the data by time
    arrange(TIME) 

In [ ]:
emp_measures$yr_quarter = as.yearqtr(emp_measures$yr_quarter)

## Checkpoint 3: Cohort-specific Job-Level measures

Now that jobs are ordered by time, we can create our two baseline measures for each job in each quarter. We generate **same_Emp** that takes value 1 if the job existed in the immediately prior quarter and is 0 otherwise. We create a similar measure, **stay_emp**, for a job in a quarter if the job existed in the quarter immediately following. It takes value 1 if this is true and 0 otherwise.

This checkpoint focuses on creating these baseline measures for the cohort of your choice. For your research project, you may be utilizing a different combination of year-quarters (**yr_quarter**). You should be able to replicate these measures for your cohort as well by updating the year-quarters for the cohort of your choice. 

In this checkpoint, we ask you update the blank with the quarter of exit you used to define your cohort.




In [ ]:
# We will use the mutate function to create new variables from the job-quarter level measures to create job-level measures
emp_measures <- emp_measures %>% 
        mutate(new_emp = ifelse(TIME == min(TIME) & yr_quarter > '___',1,0), # first record of grouping is a new job
# use the lag function to calculate how many time periods passed between two observations
           time_change_emp = TIME-lag(TIME), 
# The person has the same employer if only one time period passed between current and past observation
           same_Emp = case_when(time_change_emp == 1 ~ 1,
                                is.na(time_change_emp) ~ 0,
                                TRUE ~ 0),
# The person stayed with the same employer if only one time period passed between the next and 
# current observation
           time_change_emp2 = lead(TIME)-TIME, 
           stay_emp = case_when(time_change_emp2 == 1 ~ 1,
                               is.na(time_change_emp2) ~ 0, 
                                TRUE ~ 0)) %>%
    select(-c(time_change_emp, time_change_emp2))


In [ ]:
# Generate full employment indicator, count the number of times a job exists (number of quarters with employer), and
# count the number of full employment quarters - NO updates required
emp_measures <- emp_measures %>% mutate(full_emp = ifelse(same_Emp==1 & stay_emp == 1,1,0)) %>%
# remove TIME from the data frame
    select(-c(TIME)) %>%
    filter(yr_quarter!=min(emp_measures$yr_quarter) & yr_quarter!=max(emp_measures$yr_quarter))

head(emp_measures)

In [ ]:
# hint
check_3.hint()

In [ ]:
# solution
check_3.solution()